In [1]:
"""
create few shot (1~3) prompt from the test prompt
"""
import os
import pickle
import requests
import numpy as np
import random
import re

In [2]:
x = 128
def list_to_string(a):
    a = str(a)
    return a.replace(' ', '')

def num_to_list(num):
    return [int(x) for x in str(num)]

list_x = num_to_list(x)
print(list_x)
print(list_to_string(list_x))

[1, 2, 8]
[1,2,8]


In [3]:
def get_input_string(x,y):
    x, y = str(x), str(y)
    input_str = f'Input:\n{x}+{y}\n'

    return input_str

def get_output_string(x,y):
    x, y = str(x), str(y)

    len_x, len_y = len(x), len(y)
    list_x, list_y = num_to_list(x), num_to_list(y)

    output_str = f'Target:\n<scratch>\n'

    output_str += f'{list_to_string(list_x)} has {len_x} digits.\n'
    output_str += f'{list_to_string(list_y)} has {len_y} digits.\n'

    C=0
    A=[]
    for i in range(max(len_x, len_y)):
        a = list_x[-1] if i < len_x else 0
        b = list_y[-1] if i < len_y else 0
        c = a + b + C
        
        output_str += f'{list_to_string(list_x)} + {list_to_string(list_y)} , A={list_to_string(A)} , C={C} , {a}+{b}+{C}={c} , A->{c%10} , C->{c//10}\n'

        A.insert(0, c % 10)
        C = c // 10

        list_x = list_x[:-1]
        list_y = list_y[:-1]

    output_str += f'{list_to_string(list_x)} + {list_to_string(list_y)} , A={list_to_string(A)} C={C} , END\n</scratch>\n'
    if C == 1:
        A.insert(0, 1)
    for a in A:
        output_str += f'{a} '

    return output_str[:-1]+'\n'

In [4]:
x,y = 128, 367
input_str = get_input_string(x,y)
output_str = get_output_string(x,y)

print(input_str+output_str)
print(len(input_str+output_str))

Input:
128+367
Target:
<scratch>
[1,2,8] has 3 digits.
[3,6,7] has 3 digits.
[1,2,8] + [3,6,7] , A=[] , C=0 , 8+7+0=15 , A->5 , C->1
[1,2] + [3,6] , A=[5] , C=1 , 2+6+1=9 , A->9 , C->0
[1] + [3] , A=[9,5] , C=0 , 1+3+0=4 , A->4 , C->0
[] + [] , A=[4,9,5] C=0 , END
</scratch>
4 9 5

282


In [4]:
out_dir = 'few_shot_prefix'
num_prompts = 5

shots=1

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for num_prompt in range(1, num_prompts+1):
    output_file_path = f'{out_dir}/{shots}shot_{num_prompt}.txt'
    if os.path.exists(output_file_path):
         continue
    with open(output_file_path, 'w') as f:
        print(f'creating new file: {output_file_path}!')
        for shot in range(shots):
            x, y = random.randint(1, 1000), random.randint(1, 999)
            input_str = get_input_string(x, y)
            output_str = get_output_string(x, y)            
            f.write(input_str + output_str)


creating new file: few_shot_prefix/1shot_1.txt!
creating new file: few_shot_prefix/1shot_2.txt!
creating new file: few_shot_prefix/1shot_3.txt!
creating new file: few_shot_prefix/1shot_4.txt!
creating new file: few_shot_prefix/1shot_5.txt!


In [5]:
out_dir = 'few_shot_prefix'
num_prompts = 5

shots=2

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for num_prompt in range(1, num_prompts+1):
    output_file_path = f'{out_dir}/{shots}shot_{num_prompt}.txt'
    if os.path.exists(output_file_path):
         continue
    with open(output_file_path, 'w') as f:
        print(f'creating new file: {output_file_path}!')
        for shot in range(shots):
            x, y = random.randint(1, 1000), random.randint(1, 999)
            input_str = get_input_string(x, y)
            output_str = get_output_string(x, y)            
            f.write(input_str + output_str)


creating new file: few_shot_prefix/2shot_1.txt!
creating new file: few_shot_prefix/2shot_2.txt!
creating new file: few_shot_prefix/2shot_3.txt!
creating new file: few_shot_prefix/2shot_4.txt!
creating new file: few_shot_prefix/2shot_5.txt!


In [6]:
out_dir = 'few_shot_prefix'
num_prompts = 5

shots=3

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for num_prompt in range(1, num_prompts+1):
    output_file_path = f'{out_dir}/{shots}shot_{num_prompt}.txt'
    if os.path.exists(output_file_path):
         continue
    with open(output_file_path, 'w') as f:
        print(f'creating new file: {output_file_path}!')
        for shot in range(shots):
            x, y = random.randint(1, 1000), random.randint(1, 999)
            input_str = get_input_string(x, y)
            output_str = get_output_string(x, y)            
            f.write(input_str + output_str)


creating new file: few_shot_prefix/3shot_1.txt!
creating new file: few_shot_prefix/3shot_2.txt!
creating new file: few_shot_prefix/3shot_3.txt!
creating new file: few_shot_prefix/3shot_4.txt!
creating new file: few_shot_prefix/3shot_5.txt!


In [15]:
out_dir = 'few_shot_prompts'
num_prompts = 5

shots=3

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

input_file_path = f'prompt_addition_test_0.0001.txt'
with open(input_file_path, 'r') as f:
    lines = f.readlines()

for num_prompt in range(1, num_prompts+1):
    few_shot_prefix_path = f'few_shot_prefix/{shots}shot_{num_prompt}.txt'
    with open(few_shot_prefix_path, 'r') as f:
        data_few_shot_prefix = f.read()

    output_file_path = f'{out_dir}/prompt_addition_{shots}shot_test_0.0001_{num_prompt}.txt'
    
    print(f'creating {shots} shot prompts in {output_file_path}')

    with open(output_file_path, 'w') as f:
        total_num_examples = len(lines) // 2
        for num_ex in range(total_num_examples):
            prompt = lines[2*num_ex] + lines[2*num_ex+1]
            f.write(data_few_shot_prefix + prompt + '\n')


creating 3 shot prompts in few_shot_prompts/prompt_addition_3shot_test_0.0001_1.txt
creating 3 shot prompts in few_shot_prompts/prompt_addition_3shot_test_0.0001_2.txt
creating 3 shot prompts in few_shot_prompts/prompt_addition_3shot_test_0.0001_3.txt
creating 3 shot prompts in few_shot_prompts/prompt_addition_3shot_test_0.0001_4.txt
creating 3 shot prompts in few_shot_prompts/prompt_addition_3shot_test_0.0001_5.txt


In [16]:
out_dir = 'few_shot_prompts'
num_prompts = 5

shots=2

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

input_file_path = f'prompt_addition_test_0.0001.txt'
with open(input_file_path, 'r') as f:
    lines = f.readlines()

for num_prompt in range(1, num_prompts+1):
    few_shot_prefix_path = f'few_shot_prefix/{shots}shot_{num_prompt}.txt'
    with open(few_shot_prefix_path, 'r') as f:
        data_few_shot_prefix = f.read()

    output_file_path = f'{out_dir}/prompt_addition_{shots}shot_test_0.0001_{num_prompt}.txt'
    
    print(f'creating {shots} shot prompts in {output_file_path}')

    with open(output_file_path, 'w') as f:
        total_num_examples = len(lines) // 2
        for num_ex in range(total_num_examples):
            prompt = lines[2*num_ex] + lines[2*num_ex+1]
            f.write(data_few_shot_prefix + prompt + '\n')


creating 2 shot prompts in few_shot_prompts/prompt_addition_2shot_test_0.0001_1.txt
creating 2 shot prompts in few_shot_prompts/prompt_addition_2shot_test_0.0001_2.txt
creating 2 shot prompts in few_shot_prompts/prompt_addition_2shot_test_0.0001_3.txt
creating 2 shot prompts in few_shot_prompts/prompt_addition_2shot_test_0.0001_4.txt
creating 2 shot prompts in few_shot_prompts/prompt_addition_2shot_test_0.0001_5.txt


In [18]:
out_dir = 'few_shot_prompts'
num_prompts = 5

shots=1

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

input_file_path = f'prompt_addition_test_0.0001.txt'
with open(input_file_path, 'r') as f:
    lines = f.readlines()

for num_prompt in range(1, num_prompts+1):
    few_shot_prefix_path = f'few_shot_prefix/{shots}shot_{num_prompt}.txt'
    with open(few_shot_prefix_path, 'r') as f:
        data_few_shot_prefix = f.read()

    output_file_path = f'{out_dir}/prompt_addition_{shots}shot_test_0.0001_{num_prompt}.txt'
    
    print(f'creating {shots} shot prompts in {output_file_path}')

    with open(output_file_path, 'w') as f:
        total_num_examples = len(lines) // 2
        for num_ex in range(total_num_examples):
            prompt = lines[2*num_ex] + lines[2*num_ex+1]
            f.write(data_few_shot_prefix + prompt + '\n')


creating 1 shot prompts in few_shot_prompts/prompt_addition_1shot_test_0.0001_1.txt
creating 1 shot prompts in few_shot_prompts/prompt_addition_1shot_test_0.0001_2.txt
creating 1 shot prompts in few_shot_prompts/prompt_addition_1shot_test_0.0001_3.txt
creating 1 shot prompts in few_shot_prompts/prompt_addition_1shot_test_0.0001_4.txt
creating 1 shot prompts in few_shot_prompts/prompt_addition_1shot_test_0.0001_5.txt
